In [ ]:
!pip install datasets
!pip install torchaudio

In [ ]:
from datasets import load_dataset
import os
import torchaudio
import torchaudio.transforms as T
import torch

# Load dataset
dataset = load_dataset("DTU54DL/common-accent", split="train")

# Create output directories
os.makedirs("xtts_data/wavs", exist_ok=True)

# Define accents to keep
target_accents = {
    "India and South Asia (India, Pakistan, Sri Lanka)",
}

# Prepare metadata list
metadata = []
resampler = T.Resample(orig_freq=16000, new_freq=16000)

# Loop and filter
max_samples = 500  # Adjust as needed
count = 0

for i, sample in enumerate(dataset):
    accent = sample.get("accent", "")
    if accent not in target_accents:
        continue

    audio_array = sample["audio"]["array"]
    sentence = sample["sentence"]
    filename = f"clip_{count:05d}.wav"
    filepath = f"xtts_data/wavs/{filename}"

    # Save audio
    waveform = torch.tensor(audio_array).unsqueeze(0)
    torchaudio.save(filepath, waveform, 16000)

    metadata.append(f"{filename.replace('.wav', '')}|{sentence}")
    count += 1

    if count >= max_samples:
        break

# Save metadata
with open("xtts_data/metadata.csv", "w", encoding="utf-8") as f:
    f.write("\n".join(metadata))

print(f"✅ Done. Saved {count} samples for fine-tuning.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Done. Saved 500 samples for fine-tuning.


In [ ]:
import pandas as pd

# Load correctly with 2 columns
df = pd.read_csv("xtts_data/metadata.csv", sep="|", header=None, names=["filename", "text"])

# Add third column by duplicating 'text'
df["normalized_text"] = df["text"]

# Save the updated metadata with 3 columns
df.to_csv("xtts_data/metadata.csv", sep="|", header=False, index=False)

# Check
df_check = pd.read_csv("xtts_data/metadata.csv", sep="|", header=None)
print(df_check.head())
print(f"Number of columns: {df_check.shape[1]}")


            0                                                  1  \
0  clip_00000  Men in orange vests are at work on a construct...   
1  clip_00001                  This child is getting a pedicure.   
2  clip_00002                             Did you get the Mayor?   
3  clip_00003  Finally I didn't have to feel guilty for using...   
4  clip_00004            I'd still like to have a word with her.   

                                                   2  
0  Men in orange vests are at work on a construct...  
1                  This child is getting a pedicure.  
2                             Did you get the Mayor?  
3  Finally I didn't have to feel guilty for using...  
4            I'd still like to have a word with her.  
Number of columns: 3


In [ ]:
import os
import random

# Set your data directory and filenames
data_dir = "/content/xtts_data"
input_metadata = os.path.join(data_dir, "metadata.csv")
train_output = os.path.join(data_dir, "metadata_train.csv")
val_output = os.path.join(data_dir, "metadata_val.csv")

# Ratio to split (e.g., 90% train, 10% val)
train_ratio = 0.9

# Read the original metadata
with open(input_metadata, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Shuffle the lines randomly
random.shuffle(lines)

# Split the lines
split_index = int(len(lines) * train_ratio)
train_lines = lines[:split_index]
val_lines = lines[split_index:]

# Write to train and val files
with open(train_output, "w", encoding="utf-8") as f:
    f.writelines(train_lines)

with open(val_output, "w", encoding="utf-8") as f:
    f.writelines(val_lines)

print(f"✅ Split complete!")
print(f"  > Train samples: {len(train_lines)} → {train_output}")
print(f"  > Val samples:   {len(val_lines)} → {val_output}")


✅ Split complete!
  > Train samples: 450 → /content/xtts_data/metadata_train.csv
  > Val samples:   50 → /content/xtts_data/metadata_val.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/indicdata ./xtts_data


In [ ]:
%cd /content
!git clone -b dev https://github.com/coqui-ai/TTS.git
%cd TTS
!pip install -r requirements.txt
!pip install -e .



/content
fatal: destination path 'TTS' already exists and is not an empty directory.
/content/TTS
Ignoring numpy: markers 'python_version <= "3.10"' don't match your environment
Ignoring numba: markers 'python_version < "3.9"' don't match your environment
Obtaining file:///content/TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for TTS (pyproject.toml) ... done
  Created wheel for TTS: filename=tts-0.22.0-0.editable-cp311-cp311-linux_x86_64.whl size=15144 sha256=40def0a88ca353d39863f56b7d0087a470522abdcec45a4ce64d4b7973cda046
  Stored in directory: /tmp/pip-ephem-wheel-cache-u81v9sqy/wheels/a1/31/b6/e4589a2b0e214f7f2ecd464f78aeb7998645a43ff4721bf68b
Successfully built TTS


In [ ]:
%cd /content/


/content


In [ ]:
!pip install transformers==4.30.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires transformers>=4.33.0, but you have transformers 4.30.2 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.2 which is incompatible.


In [ ]:
!grep -r "LJSpeech" . | grep "/raid"


grep: ./TTS/tests/inputs/scale_stats.npy: binary file matches
./drive/MyDrive/Colab Notebooks/indicdataset.ipynb:{"cells":[{"cell_type":"code","execution_count":1,"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"executionInfo":{"elapsed":103526,"status":"ok","timestamp":1744443341786,"user":{"displayName":"avatar","userId":"09671676423806163443"},"user_tz":-330},"id":"Vzs4DWcoI1Wr","outputId":"9b4be41b-fdc8-4300-851a-9d10d349dd30"},"outputs":[{"output_type":"stream","name":"stdout","text":["Collecting datasets\n","  Downloading datasets-3.5.0-py3-none-any.whl.metadata (19 kB)\n","Requirement already satisfied: filelock in /usr/local/lib/python3.11/dist-packages (from datasets) (3.18.0)\n","Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.11/dist-packages (from datasets) (2.0.2)\n","Requirement already satisfied: pyarrow>=15.0.0 in /usr/local/lib/python3.11/dist-packages (from datasets) (18.1.0)\n","Collecting dill<0.3.9,>=0.3.0 (from datasets)\n","  Downl

In [ ]:
import pandas as pd

df = pd.read_csv("/content/xtts_data/metadata_val.csv", sep="|", header=None)
print(df.head())
print(f"🔍 Found {len(df)} entries in metadata_val.csv")

# Optional sanity check
import os
missing = [path for path in df[0] if not os.path.exists(os.path.join("/content/xtts_data", path))]
print("❌ Missing files:", missing)



            0                                                  1  \
0  clip_00201           Those who attempted surrender were shot.   
1  clip_00000  Men in orange vests are at work on a construct...   
2  clip_00052  Established were axle works, brickworks, and m...   
3  clip_00372  He was succeeded by his son, Nicholas, the sec...   
4  clip_00367  Gary Morrel was play-by-play announcer while N...   

                                                   2  
0           Those who attempted surrender were shot.  
1  Men in orange vests are at work on a construct...  
2  Established were axle works, brickworks, and m...  
3  He was succeeded by his son, Nicholas, the sec...  
4  Gary Morrel was play-by-play announcer while N...  
🔍 Found 50 entries in metadata_val.csv
❌ Missing files: ['clip_00201', 'clip_00000', 'clip_00052', 'clip_00372', 'clip_00367', 'clip_00069', 'clip_00432', 'clip_00478', 'clip_00318', 'clip_00277', 'clip_00027', 'clip_00144', 'clip_00030', 'clip_00185', 'clip_00

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python TTS/recipes/ljspeech/xtts_v2/train_gpt_xtts.py --config_path /content/config.json --debug
